In [1]:
!pip install easyocr pdf2image



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 20.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

In [2]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 0s (404 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.4_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.4) ...
Setting up poppler-utils (22.02.0-2ubuntu0.4) ...
Processing triggers for man-db (2.10.2-1) ...


In [3]:
import easyocr
from pdf2image import convert_from_path
from PIL import Image
import os

# Path to the PDF file
pdf_path = '/content/test.pdf'


image_dir = 'extracted_images'
os.makedirs(image_dir, exist_ok=True)


output_file = 'output.txt'


reader = easyocr.Reader(['en'])

# Convert PDF to a list of images
pages = convert_from_path(pdf_path)

with open(output_file, 'w', encoding='utf-8') as f:
    # Iterate through each page image
    for page_num, page_image in enumerate(pages):
        # Save the image for reference
        image_filename = f'page_{page_num + 1}.png'
        image_path = os.path.join(image_dir, image_filename)
        page_image.save(image_path, 'PNG')

        # Perform OCR on the image
        result = reader.readtext(image_path)

        # Write the OCR result to the output file
        f.write(f'Page {page_num + 1}\n')
        for (bbox, text, prob) in result:
            f.write(text + '\n')
        f.write('\n' + '='*80 + '\n')

print(f'Text extracted and saved to {output_file}')
print(f'Images extracted and saved in {image_dir} directory')

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Text extracted and saved to output.txt
Images extracted and saved in extracted_images directory


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import re

# Path to the uploaded text file
file_path = '/content/output.txt'

# Initialize lists to store Roll No and Name pairs
roll_name_pairs = []

# Read the text file
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Regex patterns for Roll No and Name
roll_pattern = re.compile(r'Roll No: (\d+)')
name_pattern = re.compile(r'Name: ([A-Za-z\s.]+)')

# Temporary variables to store the current Roll No and Name
current_roll = None
current_name = None

# Iterate through each line to find Roll No and Name
for line in lines:

    if "Post Name:" in line:
      continue
    roll_match = roll_pattern.search(line)
    name_match = name_pattern.search(line)

    if roll_match:
        if current_roll and not current_name:
            # If there is a current roll without a name, append null for the name
            roll_name_pairs.append((current_roll, 'null'))
        current_roll = roll_match.group(1)

    if name_match:
        if current_name and not current_roll:
            # If there is a current name without a roll, append null for the roll
            roll_name_pairs.append(('null', current_name))
        current_name = name_match.group(1)

    # If both Roll No and Name are found, add them as a pair and reset the variables
    if current_roll and current_name:
        roll_name_pairs.append((current_roll, current_name))
        current_roll = None
        current_name = None

# Handle cases where the last roll or name was found without its pair
if current_roll:
    roll_name_pairs.append((current_roll, 'null'))
elif current_name:
    roll_name_pairs.append(('null', current_name))

# Display the extracted Roll No and Name pairs
for roll, name in roll_name_pairs:
    print(f'Roll No: {roll}, Name: {name}')


Roll No: 15000101, Name: MD. SAGUR

Roll No: 15000102, Name: SAIFUL ISLAM

Roll No: 15000103, Name: FARDUSHE AKTER

Roll No: 15000104, Name: MD.ZONAYED

Roll No: 15000105, Name: MD. SAJIB MIA

Roll No: 15000106, Name: MONIR HOSSAIN

Roll No: 15000107, Name: MRIDUL TALUKDAR

Roll No: 15000108, Name: MD. SHAHINUR ISLAM

Roll No: 15000109, Name: JIHAD MOLLAH

Roll No: 15000110, Name: null
Roll No: 15000111, Name: MD. SABBIR HOSSAIN

Roll No: 15000112, Name: SABBIR AHMED KHAN

Roll No: 15000113, Name: MD EMRAN HASAN

Roll No: 15000114, Name: SUJOY SARKER

Roll No: 15000115, Name: NELOY CHOWDHURI

Roll No: 15000116, Name: RUMAN HASAN BAPPY

Roll No: 15000117, Name: RAJIB BEPARY

Roll No: 15000118, Name: REZAUL KARIM

Roll No: 15000119, Name: MD AWLAD HOSEN

Roll No: 15000120, Name: KAZI AULAD HOSSAIN

Roll No: 15000121, Name: PROBIR MONDAL

Roll No: 15000122, Name: MD. FOYZUL ISLAM

Roll No: 15000123, Name: MAHESH CHANDRA DAS

Roll No: 15000124, Name: MD. ATIKUR RAHMAN ATIIK

Roll No: 15000

In [6]:
import pandas as pd

df= pd.DataFrame(roll_name_pairs, columns=['Roll No', 'Name'])
excel_path= 'roll_name.xlsx'
df.to_excel(excel_path, index=False)



In [7]:
pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 32.6 MB/s eta 0:00:00


In [ ]:
pip install pymupdf pillow

In [8]:
pip install pdf2image Pillow opencv-python-headless


In [9]:
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [14]:
import os
import cv2
import numpy as np
from pdf2image import convert_from_path
from PIL import Image

# Path to the PDF file
pdf_path = '/content/test.pdf'

# Directory to save the extracted images
image_dir = 'extracted_images'
os.makedirs(image_dir, exist_ok=True)

# Convert PDF pages to images
pages = convert_from_path(pdf_path)

# Load the pre-trained face detector from OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize index for accessing roll_name_pairs
index = 0

# Desired size for the resized images
desired_size = (300, 300)  # You can adjust this size as needed

# Variable to count total number of detected faces
total_faces_detected = 0

# Padding factor
padding = 50  # Adjust this value as needed

# Iterate through each page image
for page_num, page_image in enumerate(pages):
    # Convert PIL image to OpenCV format
    open_cv_image = np.array(page_image)
    open_cv_image = open_cv_image[:, :, ::-1].copy()

    # Convert to grayscale
    gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=25, minSize=(60, 60))
    print(f"Page Num: {page_num+1}, face count: {len(faces)}")

    # Count the number of faces detected on this page
    num_faces = len(faces)
    total_faces_detected += num_faces

    # Extract and save each face
    for i, (x, y, w, h) in enumerate(faces):
        # Calculate padding
        x1 = max(x - padding, 0)
        y1 = max(y - padding, 0)
        x2 = min(x + w + padding, open_cv_image.shape[1])
        y2 = min(y + h + padding, open_cv_image.shape[0])

        roi = open_cv_image[y1:y2, x1:x2]
        roi_image = Image.fromarray(cv2.cvtColor(roi, cv2.COLOR_BGR2RGB))

        # Resize the extracted face image to the desired size
        # roi_image = roi_image.resize(desired_size, Image.ANTIALIAS)

        if index < len(roll_name_pairs):
            roll, name = roll_name_pairs[index]
            index += 1
        else:
            roll, name = 'unknown', 'unknown'

        # Save the extracted face image
        image_filename = f'{roll}_{name.replace(" ", "_")}.jpg'
        image_path = os.path.join(image_dir, image_filename)
        roi_image.save(image_path)

# Print the total number of faces detected
print(f'Total faces detected: {total_faces_detected}')


Page Num: 1, face count: 2
Page Num: 2, face count: 3
Page Num: 3, face count: 3
Page Num: 4, face count: 3
Page Num: 5, face count: 3
Page Num: 6, face count: 3
Page Num: 7, face count: 3
Page Num: 8, face count: 3
Page Num: 9, face count: 3
Page Num: 10, face count: 3
Page Num: 11, face count: 3
Page Num: 12, face count: 3
Page Num: 13, face count: 2
Page Num: 14, face count: 4
Page Num: 15, face count: 3
Page Num: 16, face count: 3
Page Num: 17, face count: 3
Page Num: 18, face count: 3
Page Num: 19, face count: 3
Page Num: 20, face count: 3
Page Num: 21, face count: 3
Page Num: 22, face count: 3
Page Num: 23, face count: 3
Page Num: 24, face count: 3
Page Num: 25, face count: 3
Page Num: 26, face count: 3
Page Num: 27, face count: 3
Page Num: 28, face count: 3
Page Num: 29, face count: 3
Page Num: 30, face count: 3
Page Num: 31, face count: 3
Page Num: 32, face count: 3
Page Num: 33, face count: 3
Page Num: 34, face count: 2
Total faces detected: 100


In [ ]:
!zip -r /content/images.zip /content/extracted_images

  adding: content/extracted_images/ (stored 0%)
  adding: content/extracted_images/15000194_BAYEJID^J.jpg (deflated 13%)
  adding: content/extracted_images/15000154_MD_JAZE_MIA^J.jpg (deflated 7%)
  adding: content/extracted_images/page_34.png (deflated 6%)
  adding: content/extracted_images/15000176_ABUL_KHAIR_MOHAMMAD_IKRAM^J.jpg (deflated 8%)
  adding: content/extracted_images/15000147_MD_SOJON_HOSSAIN^J.jpg (deflated 10%)
  adding: content/extracted_images/15000162_MD._SALMAN_HASAN^J.jpg (deflated 4%)
  adding: content/extracted_images/page_11.png (deflated 6%)
  adding: content/extracted_images/page_27.png (deflated 5%)
  adding: content/extracted_images/15000178_MD.SUMON_MIA^J.jpg (deflated 5%)
  adding: content/extracted_images/15000157_MD._RAKIBUL_HASSAN^J.jpg (deflated 12%)
  adding: content/extracted_images/15000113_MD_EMRAN_HASAN^J.jpg (deflated 7%)
  adding: content/extracted_images/15000190_MD._RIYAD_TALUKDAR^J.jpg (deflated 8%)
  adding: content/extracted_images/15000132_